The following is a data science illustration of The foundational paper on FDFPGs. This is meant to be part one of a two part paper review, with a future modeling of Quadratic Funding and Matching Funds Requirments by Pasquini at Gitcoin. 

This worik is for educational purposes in the domain of token engineering. This work is meant to illustrate a data science approach to systems based modeling. This work is in the domain of digital signal processing, with primary foundations in machine learning, big data, data science, economics, and computing science, as those are the backgrounds of the primary authors. This work is funded by  with the Token Engineering Commons and Griff Green. 

# A Flexible Design for Funding Public Goods

Bhuterin, Hitzig, Weyl on A Flexible Design for Funding Public Goods. 

https://arxiv.org/pdf/1809.06421.pdf

The following is a data science illustration of The foundational paper on FDFPGs. This is meant to be part one of a two part paper review, with a future modeling of Quadratic Funding and Matching Funds Requirments by Pasquini at Gitcoin. 

This worik is for educational purposes in the domain of token engineering. This work is meant to illustrate a data science approach to systems based modeling. This work is in the domain of digital signal processing, with primary foundations in machine learning, big data, data science, economics, and computing science, as those are the backgrounds of the primary authors. This work is funded by  with the Token Engineering Commons and Griff Green. 

## 3.0 Model

* The Foundational Model
* Society, Citizens, Community, and Public Goods.

In [ ]:
import numpy as np

# Number of Citizens in the Society
N = 30

# Max Community Size
C = 20

# Max Number of Public Goods
P = 10

# Society is a set of citizens
society = list(range(N))

# Community is a random subset of the society. The community size is from 25 up to size of the society.
community = np.random.choice(a=list(society), size=C, replace=False, p=None)

# Public Goods are proposed by community members. Cardinality is from 20 up to size of the community.
public_goods = list(enumerate(np.random.choice(a=list(community), size=P, replace=True, p=None)))

Community members are sampled from society.

In [ ]:
community

array([26, 29,  8, 11,  6,  5,  1,  3, 13, 10, 15, 17, 14,  7, 19,  0, 25,
       28, 24, 12])

Public goods are sampled from the community with replace=True ie. a community member can steward multiple public goods.

In [ ]:
public_goods

[(0, 8),
 (1, 15),
 (2, 0),
 (3, 12),
 (4, 0),
 (5, 7),
 (6, 6),
 (7, 6),
 (8, 26),
 (9, 11)]

In [ ]:
len(society)

30

In [ ]:
len(community)

20

In [ ]:
len(public_goods)

10

## 3.1 Individual Preferences and Actions
* Value Function Generators
* Value Function Dataset
* Visualizing Value Functions
* Public Goods Generators
* Citizen Contribution Generators
* The Contributions Dataset
* The Utility of Citizens

### Value Function Generators

The paper calls for 
* Concave, 
* Smooth, 
* Increasing preferences towards the funding of public goods.

The following function generator, generates value functions in the domain of {[0,1],[0,1]}.

In [ ]:
import param
import numpy as np
import panel as pn
import hvplot.pandas
import pandas as pd

class ConcaveFunctionGenerator(param.Parameterized):
    f0 = param.Number(default=0.2, bounds=(0, 1), doc="Value of f(0)")
    f1 = param.Number(default=0.8, bounds=(0, 1), softbounds=(0, 1), doc="Value of f(1)")
    steepness = param.Number(default=5, bounds=(1, 20), doc="Steepness of the curve")

    def __init__(self, **params):
        super().__init__(**params)
        self._update_f1_bounds()

    @param.depends('f0', watch=True)
    def _update_f1_bounds(self):
        # Clip the value of f1 if it's below f0
        self.f1 = max(self.f0, self.f1)
        
        # Update the lower bound of f1 to be the value of f0
        self.param['f1'].bounds = (self.f0, 1)
        
    def x(self):
        return np.linspace(0, 1, 400)

    @param.depends('f0', 'f1', 'steepness')
    def f(self, x):
        # Using the negative exponential function as a base
        y = 1 - np.exp(-self.steepness * x)
        
        # Adjusting the function to start at f0 and end at f1
        y = self.f0 + (self.f1 - self.f0) * (y - y.min()) / (y.max() - y.min())
        
        return y

    @param.depends('f0', 'f1', 'steepness')
    def view(self):
        x = self.x()
        y = self.f(x)
        df = pd.DataFrame({'x': x, 'y': y})
        return df.hvplot.line(x='x', y='y', ylim=(0, 1.01))

concave_gen = ConcaveFunctionGenerator()
pn.Row(concave_gen.param, concave_gen.view).servable()


Row
    [0] Column(margin=(5, 10), name='ConcaveFunctionGenerator'...)
        [0] StaticText(value='<b>ConcaveFunctionGenerat...)
        [1] FloatSlider(name='F0', value=0.2)
        [2] FloatSlider(name='F1', start=0.2, value=0.8)
        [3] FloatSlider(end=20, name='Steepness', start=1, value=5)
    [1] ParamMethod(method, _pane=HoloViews, defer_load=False)

The concave value generator has the following properties:
1. f0 = f(0) is in [0,1]
2. f1 = f(1) is in [f0,1]
3. f(x) = 1 - e**(-s*x)
4. The negative exponential function provides a smooth, concave, increasing function from f0 to f1 with a parameterized steepness.

The class dynamically clips f1 to satisfy property 2. Such as in the following example:

In [ ]:
ConcaveFunctionGenerator(f0=1,f1=0)

ConcaveFunctionGenerator(f0=1, f1=1, name='ConcaveFunctionGenerator00955', steepness=5)

Here we propose parameter generators for sampling concave functions

In [ ]:
import numbergen as ng
import numpy as np


# For CurveGenerator
def concave_function_parameters_generator():
    return dict(
        f0=ng.BoundedNumber(generator=ng.NormalRandom(mu=0.1, sigma=0.3), bounds=(0,1))(),
        f1=ng.BoundedNumber(generator=ng.NormalRandom(mu=0.5, sigma=0.4), bounds=(0,1))(),
        steepness=ng.UniformRandom(lbound=1, ubound=20)(),
    )

This method generates random parameters given the distributions described for f0, f1, and steepness above. 
* f0 is a normal distribution at (0.1,0.3), clipped at (0,1)
* f0 is a normal distribution at (0.5,0.4), clipped at (0,1)
* steepness is a uniform distribution at (1,20)

In [ ]:
concave_function_parameters_generator()

{'f0': 0.4736495272194561,
 'f1': 0.3104704632254229,
 'steepness': 8.166623864997234}

In [ ]:
value_functions = [ConcaveFunctionGenerator(**concave_function_parameters_generator()) for p_i in range(len(public_goods)*len(society))]

Here we can see all of the preferences of the citizens. These are the value functions V_p_i(F_p). There is one preference function for each V_p_i in {public_goods}X{citizens}. Cardinality is len(public_goods)*len(citizens).

#### Visualizing Sampled Value Functions

In [ ]:
import pandas as pd

pd.DataFrame([s.param.values() for s in value_functions])

sample_p_i_slider = pn.widgets.IntSlider(name='Utility Value Function', start=0, end=len(value_functions)-1)

pn.Row(sample_p_i_slider, pn.bind(lambda i: value_functions[i].view(), i=sample_p_i_slider))

Row
    [0] IntSlider(end=299, name='Utility Value Function')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False)

#### Value Functions Dataset

Create a Dataframe! These are citizen preferences by funding amounts for public goods. Take Funding Amount as the Index by applying a transpose. 

In [ ]:
df_value_functions = pd.DataFrame([s.f(s.x()) for s in value_functions])
df_value_functions = df_value_functions.T
df_value_functions.shape

(400, 300)

Make Funding the Index from 0 to 100

In [ ]:
df_value_functions.index = np.linspace(0,1,len(df_value_functions))
df_value_functions.index.name = "funding"

Label the Columns by {public_goods}X{Citizens}.

In [ ]:
df_value_functions.columns = [(p, i) for p in public_goods for i in society]
df_value_functions.columns.name = "value_p_i"

Number of columns is len(public_goods)*len(society).

In [ ]:
df_value_functions

value_p_i,"((0, 8), 0)","((0, 8), 1)","((0, 8), 2)","((0, 8), 3)","((0, 8), 4)","((0, 8), 5)","((0, 8), 6)","((0, 8), 7)","((0, 8), 8)","((0, 8), 9)",...,"((9, 11), 20)","((9, 11), 21)","((9, 11), 22)","((9, 11), 23)","((9, 11), 24)","((9, 11), 25)","((9, 11), 26)","((9, 11), 27)","((9, 11), 28)","((9, 11), 29)"
funding,,,,,,,,,,,,,,,,,,,,,
0.000000,0.343669,0.012858,0.000000,0.246948,0.201957,0.209610,0.000000,0.325468,0.000000,0.000000,...,0.442816,0.571389,0.000000,0.265514,0.000000,0.435783,0.000000,0.000000,0.189957,0.000000
0.002506,0.352890,0.012858,0.017675,0.259983,0.213177,0.220077,0.015651,0.332300,0.018156,0.021948,...,0.442816,0.583684,0.002526,0.298901,0.016839,0.435783,0.003556,0.040802,0.216055,0.009678
0.005013,0.361969,0.012858,0.035016,0.272467,0.224120,0.230060,0.030764,0.338899,0.035954,0.043414,...,0.442816,0.595426,0.005035,0.330769,0.033394,0.435783,0.007098,0.079754,0.241267,0.019184
0.007519,0.370907,0.012858,0.052030,0.284423,0.234792,0.239580,0.045358,0.345274,0.053399,0.064409,...,0.442816,0.606638,0.007528,0.361189,0.049672,0.435783,0.010627,0.116940,0.265624,0.028523
0.010025,0.379708,0.012858,0.068723,0.295874,0.245200,0.248659,0.059451,0.351431,0.070500,0.084943,...,0.442816,0.617345,0.010004,0.390227,0.065675,0.435783,0.014142,0.152440,0.289156,0.037696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.989975,0.940220,0.012858,0.936109,0.555290,0.655560,0.435623,0.455713,0.525986,0.918183,0.999987,...,0.442816,0.844156,0.350400,1.000000,0.999919,0.435783,0.726674,0.899941,0.959485,0.547111
0.992481,0.940240,0.012858,0.936118,0.555290,0.655560,0.435623,0.455713,0.525986,0.918189,0.999990,...,0.442816,0.844156,0.350577,1.000000,0.999940,0.435783,0.727461,0.899941,0.959485,0.547120
0.994987,0.940260,0.012858,0.936128,0.555290,0.655561,0.435623,0.455713,0.525986,0.918196,0.999994,...,0.442816,0.844156,0.350754,1.000000,0.999960,0.435783,0.728245,0.899941,0.959485,0.547128


In [ ]:
df_value_functions_melted = df_value_functions.melt(ignore_index=False)
df_value_functions_melted['public_good'] = df_value_functions_melted['value_p_i'].astype(str).apply(eval).apply(lambda x: x[0]).astype(str)
df_value_functions_melted['citizen'] = df_value_functions_melted['value_p_i'].astype(str).apply(eval).apply(lambda x: x[1]).astype(str)
df_value_functions_melted

,value_p_i,value,public_good,citizen
funding,,,,
0.000000,"((0, 8), 0)",0.343669,"(0, 8)",0
0.002506,"((0, 8), 0)",0.352890,"(0, 8)",0
0.005013,"((0, 8), 0)",0.361969,"(0, 8)",0
0.007519,"((0, 8), 0)",0.370907,"(0, 8)",0
0.010025,"((0, 8), 0)",0.379708,"(0, 8)",0
...,...,...,...,...
0.989975,"((9, 11), 29)",0.547111,"(9, 11)",29
0.992481,"((9, 11), 29)",0.547120,"(9, 11)",29
0.994987,"((9, 11), 29)",0.547128,"(9, 11)",29


#### V_p(F_p)

In [ ]:
vpfp = df_value_functions_melted.groupby(['funding', 'public_good'])[['value']].sum().reset_index()

In [ ]:
vpfp

,funding,public_good,value
0,0.0,"(0, 8)",4.393434
1,0.0,"(1, 15)",5.722815
2,0.0,"(2, 0)",4.919889
3,0.0,"(3, 12)",6.437635
4,0.0,"(4, 0)",5.463763
...,...,...,...
3995,1.0,"(5, 7)",19.269947
3996,1.0,"(6, 6)",17.435438
3997,1.0,"(7, 6)",17.213092
3998,1.0,"(8, 26)",17.117675


In [ ]:
vpfp = vpfp.pivot_table(columns='public_good', values='value', index='funding')
vpfp

public_good,"(0, 8)","(1, 15)","(2, 0)","(3, 12)","(4, 0)","(5, 7)","(6, 6)","(7, 6)","(8, 26)","(9, 11)"
funding,,,,,,,,,,
0.000000,4.393434,5.722815,4.919889,6.437635,5.463763,5.764467,4.822117,4.707786,5.590134,4.736193
0.002506,4.776719,5.992478,5.174256,6.775899,5.688858,6.104006,5.175196,5.023228,5.877154,5.163410
0.005013,5.147183,6.252998,5.421210,7.103134,5.907008,6.432986,5.516160,5.328753,6.154978,5.575383
0.007519,5.505313,6.504715,5.660996,7.419744,6.118467,6.751779,5.845466,5.624712,6.423945,5.972715
0.010025,5.851575,6.747956,5.893849,7.726117,6.323482,7.060742,6.163552,5.911443,6.684379,6.355983
...,...,...,...,...,...,...,...,...,...,...
0.989975,18.325229,15.375164,15.728648,19.304529,15.766816,19.261361,17.430367,17.207848,17.111313,20.111893
0.992481,18.325847,15.376090,15.730077,19.306015,15.768932,19.263520,17.431647,17.209172,17.112918,20.114237
0.994987,18.326460,15.377010,15.731498,19.307493,15.771035,19.265670,17.432919,17.210487,17.114513,20.116567


In [ ]:
marginal_value = vpfp.diff().div(vpfp.index.to_series().diff(), axis=0).bfill()
marginal_value

public_good,"(0, 8)","(1, 15)","(2, 0)","(3, 12)","(4, 0)","(5, 7)","(6, 6)","(7, 6)","(8, 26)","(9, 11)"
funding,,,,,,,,,,
0.000000,152.930498,107.595527,101.492261,134.967455,89.812783,135.475946,140.878388,125.861355,114.520755,170.459409
0.002506,152.930498,107.595527,101.492261,134.967455,89.812783,135.475946,140.878388,125.861355,114.520755,170.459409
0.005013,147.815237,103.947494,98.534661,130.566708,87.041846,131.263234,136.044650,121.904539,110.852044,164.377377
0.007519,142.893887,100.435108,95.674553,126.327317,84.372475,127.198533,131.393130,118.087659,107.317889,158.535494
0.010025,138.158527,97.053052,92.908469,122.242950,81.800628,123.276203,126.916607,114.405427,103.912945,152.923816
...,...,...,...,...,...,...,...,...,...,...
0.989975,0.248569,0.372015,0.573352,0.595994,0.849405,0.864398,0.514294,0.531808,0.644412,0.940159
0.992481,0.246470,0.369510,0.570150,0.592857,0.844251,0.861177,0.510905,0.528300,0.640453,0.934996
0.994987,0.244392,0.367024,0.566971,0.589741,0.839132,0.857974,0.507541,0.524821,0.636521,0.929867


In [ ]:
positive_marginal_value = marginal_value.where(marginal_value > 1, 0)
positive_marginal_value

public_good,"(0, 8)","(1, 15)","(2, 0)","(3, 12)","(4, 0)","(5, 7)","(6, 6)","(7, 6)","(8, 26)","(9, 11)"
funding,,,,,,,,,,
0.000000,152.930498,107.595527,101.492261,134.967455,89.812783,135.475946,140.878388,125.861355,114.520755,170.459409
0.002506,152.930498,107.595527,101.492261,134.967455,89.812783,135.475946,140.878388,125.861355,114.520755,170.459409
0.005013,147.815237,103.947494,98.534661,130.566708,87.041846,131.263234,136.044650,121.904539,110.852044,164.377377
0.007519,142.893887,100.435108,95.674553,126.327317,84.372475,127.198533,131.393130,118.087659,107.317889,158.535494
0.010025,138.158527,97.053052,92.908469,122.242950,81.800628,123.276203,126.916607,114.405427,103.912945,152.923816
...,...,...,...,...,...,...,...,...,...,...
0.989975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.992481,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.994987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
individual_marginal_value = df_value_functions.diff().div(df_value_functions.index.to_series().diff(), axis=0).bfill()
individual_marginal_value

value_p_i,"((0, 8), 0)","((0, 8), 1)","((0, 8), 2)","((0, 8), 3)","((0, 8), 4)","((0, 8), 5)","((0, 8), 6)","((0, 8), 7)","((0, 8), 8)","((0, 8), 9)",...,"((9, 11), 20)","((9, 11), 21)","((9, 11), 22)","((9, 11), 23)","((9, 11), 24)","((9, 11), 25)","((9, 11), 26)","((9, 11), 27)","((9, 11), 28)","((9, 11), 29)"
funding,,,,,,,,,,,,,,,,,,,,,
0.000000,3.679054,0.0,7.052265,5.201076e+00,4.476913,4.176425e+00,6.244650,2.725966,7.244369,8.757099,...,0.0,4.906052e+00,1.007956,1.332109e+01,6.718686,0.0,1.418723,1.627991e+01,10.412847,3.861380
0.002506,3.679054,0.0,7.052265,5.201076e+00,4.476913,4.176425e+00,6.244650,2.725966,7.244369,8.757099,...,0.0,4.906052e+00,1.007956,1.332109e+01,6.718686,0.0,1.418723,1.627991e+01,10.412847,3.861380
0.005013,3.622310,0.0,6.919181,4.981199e+00,4.366178,3.983004e+00,6.030188,2.633087,7.101172,8.564929,...,0.0,4.684896e+00,1.001199,1.271558e+01,6.605681,0.0,1.413318,1.554181e+01,10.059712,3.793137
0.007519,3.566442,0.0,6.788609,4.770617e+00,4.258181,3.798542e+00,5.823091,2.543374,6.960805,8.376976,...,0.0,4.473710e+00,0.994487,1.213759e+01,6.494577,0.0,1.407933,1.483717e+01,9.718553,3.726100
0.010025,3.511436,0.0,6.660501,4.568937e+00,4.152856,3.622622e+00,5.623106,2.456716,6.823212,8.193148,...,0.0,4.272043e+00,0.987821,1.158587e+01,6.385342,0.0,1.402570,1.416448e+01,9.388963,3.660248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.989975,0.008055,0.0,0.003876,2.113071e-07,0.000232,3.212282e-08,0.000007,0.000003,0.002778,0.001398,...,0.0,6.281291e-08,0.071205,1.459775e-07,0.008410,0.0,0.315333,1.872234e-07,0.000013,0.003432
0.992481,0.007931,0.0,0.003803,2.023740e-07,0.000226,3.063512e-08,0.000006,0.000003,0.002723,0.001368,...,0.0,5.998148e-08,0.070728,1.393421e-07,0.008269,0.0,0.314132,1.787350e-07,0.000013,0.003372
0.994987,0.007808,0.0,0.003731,1.938185e-07,0.000221,2.921635e-08,0.000006,0.000003,0.002670,0.001338,...,0.0,5.727754e-08,0.070254,1.330083e-07,0.008129,0.0,0.312935,1.706315e-07,0.000012,0.003312


In [ ]:
positive_individual_marginal_value = individual_marginal_value.where(individual_marginal_value > 1, 0)
positive_individual_marginal_value

value_p_i,"((0, 8), 0)","((0, 8), 1)","((0, 8), 2)","((0, 8), 3)","((0, 8), 4)","((0, 8), 5)","((0, 8), 6)","((0, 8), 7)","((0, 8), 8)","((0, 8), 9)",...,"((9, 11), 20)","((9, 11), 21)","((9, 11), 22)","((9, 11), 23)","((9, 11), 24)","((9, 11), 25)","((9, 11), 26)","((9, 11), 27)","((9, 11), 28)","((9, 11), 29)"
funding,,,,,,,,,,,,,,,,,,,,,
0.000000,3.679054,0.0,7.052265,5.201076,4.476913,4.176425,6.244650,2.725966,7.244369,8.757099,...,0.0,4.906052,1.007956,13.321093,6.718686,0.0,1.418723,16.279913,10.412847,3.861380
0.002506,3.679054,0.0,7.052265,5.201076,4.476913,4.176425,6.244650,2.725966,7.244369,8.757099,...,0.0,4.906052,1.007956,13.321093,6.718686,0.0,1.418723,16.279913,10.412847,3.861380
0.005013,3.622310,0.0,6.919181,4.981199,4.366178,3.983004,6.030188,2.633087,7.101172,8.564929,...,0.0,4.684896,1.001199,12.715580,6.605681,0.0,1.413318,15.541810,10.059712,3.793137
0.007519,3.566442,0.0,6.788609,4.770617,4.258181,3.798542,5.823091,2.543374,6.960805,8.376976,...,0.0,4.473710,0.000000,12.137591,6.494577,0.0,1.407933,14.837171,9.718553,3.726100
0.010025,3.511436,0.0,6.660501,4.568937,4.152856,3.622622,5.623106,2.456716,6.823212,8.193148,...,0.0,4.272043,0.000000,11.585874,6.385342,0.0,1.402570,14.164479,9.388963,3.660248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.989975,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
0.992481,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
0.994987,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [ ]:
individual_marginal_value_melted = positive_individual_marginal_value.melt(ignore_index=False)
individual_marginal_value_melted['public_good'] = individual_marginal_value_melted['value_p_i'].astype(str).apply(eval).apply(lambda x: x[0]).astype(str)
individual_marginal_value_melted['citizen'] = individual_marginal_value_melted['value_p_i'].astype(str).apply(eval).apply(lambda x: x[1]).astype(str)
individual_marginal_value_melted

,value_p_i,value,public_good,citizen
funding,,,,
0.000000,"((0, 8), 0)",3.679054,"(0, 8)",0
0.002506,"((0, 8), 0)",3.679054,"(0, 8)",0
0.005013,"((0, 8), 0)",3.622310,"(0, 8)",0
0.007519,"((0, 8), 0)",3.566442,"(0, 8)",0
0.010025,"((0, 8), 0)",3.511436,"(0, 8)",0
...,...,...,...,...
0.989975,"((9, 11), 29)",0.000000,"(9, 11)",29
0.992481,"((9, 11), 29)",0.000000,"(9, 11)",29
0.994987,"((9, 11), 29)",0.000000,"(9, 11)",29


In [ ]:
individual_marginal_value = individual_marginal_value_melted.pivot_table(index='funding', columns='public_good', values='value', aggfunc='sum')
individual_marginal_value

public_good,"(0, 8)","(1, 15)","(2, 0)","(3, 12)","(4, 0)","(5, 7)","(6, 6)","(7, 6)","(8, 26)","(9, 11)"
funding,,,,,,,,,,
0.000000,150.918059,105.231974,101.043691,134.437100,86.400709,134.071340,139.318471,124.771714,112.392444,168.925155
0.002506,150.918059,105.231974,101.043691,134.437100,86.400709,134.071340,139.318471,124.771714,112.392444,168.925155
0.005013,145.845790,101.647822,98.091508,130.038216,83.661209,129.893145,134.499577,120.839739,108.775844,161.862927
0.007519,140.966177,98.197405,94.242621,125.800683,81.022881,125.861748,129.862750,116.049445,105.292286,155.093959
0.010025,136.271341,94.875464,91.498727,121.718166,78.481692,121.971560,125.400770,112.431289,101.936474,149.553698
...,...,...,...,...,...,...,...,...,...,...
0.989975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.992481,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.994987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
take_positive_index = lambda marginal_value: marginal_value.apply(lambda col: col[col != 0].last_valid_index()).replace(np.nan, 0)

In [ ]:
optimal_funding = take_positive_index(positive_marginal_value)
optimal_funding

public_good
(0, 8)     0.641604
(1, 15)    0.651629
(2, 0)     0.759398
(3, 12)    0.766917
(4, 0)     0.922306
(5, 7)     0.894737
(6, 6)     0.749373
(7, 6)     0.766917
(8, 26)    0.814536
(9, 11)    0.959900
dtype: float64

In [ ]:
private_contributions_funding = take_positive_index(individual_marginal_value)
private_contributions_funding

public_good
(0, 8)     0.258145
(1, 15)    0.370927
(2, 0)     0.390977
(3, 12)    0.275689
(4, 0)     0.343358
(5, 7)     0.388471
(6, 6)     0.401003
(7, 6)     0.290727
(8, 26)    0.363409
(9, 11)    0.385965
dtype: float64

In [ ]:
positive_marginal_value.hvplot.area()

:NdOverlay   [public_good]
   :Area   [funding]   (value,Baseline)

In [ ]:
individual_marginal_value.hvplot.area()

:NdOverlay   [public_good]
   :Area   [funding]   (value,Baseline)

In [ ]:
positive_marginal_value.sum(axis=1).hvplot.line(label='Collective Marginal Value', title="Suboptimality of Private Contributions") * individual_marginal_value.sum(axis=1).hvplot.line(label='Private Marginal Value')

:Overlay
   .Curve.Collective_Marginal_Value :Curve   [funding]   (0)
   .Curve.Private_Marginal_Value    :Curve   [funding]   (0)

### Definition 4 (1P1V Mechanisms)
Majority voting determines whether to fund each public good and the goods selected receive funding through taxes and transfers.

In [ ]:
individual_marginal_value

public_good,"(0, 8)","(1, 15)","(2, 0)","(3, 12)","(4, 0)","(5, 7)","(6, 6)","(7, 6)","(8, 26)","(9, 11)"
funding,,,,,,,,,,
0.000000,150.918059,105.231974,101.043691,134.437100,86.400709,134.071340,139.318471,124.771714,112.392444,168.925155
0.002506,150.918059,105.231974,101.043691,134.437100,86.400709,134.071340,139.318471,124.771714,112.392444,168.925155
0.005013,145.845790,101.647822,98.091508,130.038216,83.661209,129.893145,134.499577,120.839739,108.775844,161.862927
0.007519,140.966177,98.197405,94.242621,125.800683,81.022881,125.861748,129.862750,116.049445,105.292286,155.093959
0.010025,136.271341,94.875464,91.498727,121.718166,78.481692,121.971560,125.400770,112.431289,101.936474,149.553698
...,...,...,...,...,...,...,...,...,...,...
0.989975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.992481,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.994987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
vote_results = individual_marginal_value_melted.groupby(['funding', 'public_good'])['value'].median().to_frame().pivot_table(index='funding', columns='public_good', values='value')
vote_results

public_good,"(0, 8)","(1, 15)","(2, 0)","(3, 12)","(4, 0)","(5, 7)","(6, 6)","(7, 6)","(8, 26)","(9, 11)"
funding,,,,,,,,,,
0.000000,4.054433,1.643234,1.484398,4.066893,1.128875,2.869853,3.485239,2.024395,1.568074,4.169483
0.002506,4.054433,1.643234,1.484398,4.066893,1.128875,2.869853,3.485239,2.024395,1.568074,4.169483
0.005013,3.884398,1.612103,1.467463,3.984028,1.116350,2.817865,3.408183,1.997891,1.554474,4.041349
0.007519,3.721575,1.581761,1.451966,3.902860,1.104011,2.766820,3.325961,1.971739,1.541006,3.917766
0.010025,3.567029,1.552186,1.436662,3.797340,1.091855,2.705276,3.256950,1.945933,1.527669,3.798558
...,...,...,...,...,...,...,...,...,...,...
0.989975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.992481,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.994987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### Vote Results

In [ ]:
(vote_results > 1).astype(int)

public_good,"(0, 8)","(1, 15)","(2, 0)","(3, 12)","(4, 0)","(5, 7)","(6, 6)","(7, 6)","(8, 26)","(9, 11)"
funding,,,,,,,,,,
0.000000,1,1,1,1,1,1,1,1,1,1
0.002506,1,1,1,1,1,1,1,1,1,1
0.005013,1,1,1,1,1,1,1,1,1,1
0.007519,1,1,1,1,1,1,1,1,1,1
0.010025,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
0.989975,0,0,0,0,0,0,0,0,0,0
0.992481,0,0,0,0,0,0,0,0,0,0
0.994987,0,0,0,0,0,0,0,0,0,0


In [ ]:
(positive_marginal_value * (vote_results > 1).astype(int)).sum(axis=1).hvplot.line(label="1p1v") * positive_marginal_value.sum(axis=1).hvplot.line(label="Optimal", title="Marginal Value of 1p1v vs optimal.")

:Overlay
   .Curve.A_1p1v  :Curve   [funding]   (0)
   .Curve.Optimal :Curve   [funding]   (0)